In [1]:
from models import load_cddd_encoder

import torch
import _codecs
import numpy as np

# torch.serialization.add_safe_globals([np.dtypes.Int64DType, _codecs.encode, np.core.multiarray.scalar])


# cddd_enc = load_cddd_encoder('1k_test/models/cddd_encoder.pt')
cddd_enc = load_cddd_encoder('600k_chembl_float16/models/cddd_encoder.pt', weights_only=False)
cddd_enc

CDDDEncoder(
  (encoder): Encoder(
    (gru1): GRU(56, 512, batch_first=True)
    (gru2): GRU(512, 1024, batch_first=True)
    (gru3): GRU(1024, 2048, batch_first=True)
    (dropout): Dropout(p=0.15, inplace=False)
    (latent_projection): Linear(in_features=3584, out_features=512, bias=True)
    (gaussian_noise): GaussianNoise()
  )
)

In [2]:
for k,v in cddd_enc.encoder.state_dict().items():
    print(k, v.dtype)
# cddd_enc.encoder.to(torch.bfloat16)

gru1.weight_ih_l0 torch.bfloat16
gru1.weight_hh_l0 torch.bfloat16
gru1.bias_ih_l0 torch.bfloat16
gru1.bias_hh_l0 torch.bfloat16
gru2.weight_ih_l0 torch.bfloat16
gru2.weight_hh_l0 torch.bfloat16
gru2.bias_ih_l0 torch.bfloat16
gru2.bias_hh_l0 torch.bfloat16
gru3.weight_ih_l0 torch.bfloat16
gru3.weight_hh_l0 torch.bfloat16
gru3.bias_ih_l0 torch.bfloat16
gru3.bias_hh_l0 torch.bfloat16
latent_projection.weight torch.bfloat16
latent_projection.bias torch.bfloat16


In [5]:
cddd_enc(['CO', 'CN'], verbose=True)

Encoding   SMILES: 100%|██████████| 1/1 [00:00<00:00, 154.09it/s]


array([[ 0.453125  , -0.9140625 , -0.421875  , ...,  0.78125   ,
         0.875     , -0.8203125 ],
       [ 0.44921875, -0.9375    , -0.3046875 , ...,  0.82421875,
         0.83984375, -0.7734375 ]], dtype=float32)

In [9]:
import torch

checkpoint = torch.load("600k_chembl_float16/checkpoints/last.ckpt", map_location="cpu", weights_only=True)
for key, val in checkpoint.items():
    print(key, val.dtype if hasattr(val, 'dtype') else type(val))
    # checkpoint[key] = checkpoint[key].to(torch.bfloat16)
# model.load_state_dict(checkpoint)

epoch <class 'int'>
global_step <class 'int'>
pytorch-lightning_version <class 'str'>
state_dict <class 'collections.OrderedDict'>
loops <class 'dict'>
callbacks <class 'dict'>
optimizer_states <class 'list'>
lr_schedulers <class 'list'>
hparams_name <class 'str'>
hyper_parameters <class 'dict'>


In [13]:
for k, v in checkpoint['state_dict'].items():
    print(k, v.dtype if hasattr(v, 'dtype') else type(v))

model.encoder.gru1.weight_ih_l0 torch.bfloat16
model.encoder.gru1.weight_hh_l0 torch.bfloat16
model.encoder.gru1.bias_ih_l0 torch.bfloat16
model.encoder.gru1.bias_hh_l0 torch.bfloat16
model.encoder.gru2.weight_ih_l0 torch.bfloat16
model.encoder.gru2.weight_hh_l0 torch.bfloat16
model.encoder.gru2.bias_ih_l0 torch.bfloat16
model.encoder.gru2.bias_hh_l0 torch.bfloat16
model.encoder.gru3.weight_ih_l0 torch.bfloat16
model.encoder.gru3.weight_hh_l0 torch.bfloat16
model.encoder.gru3.bias_ih_l0 torch.bfloat16
model.encoder.gru3.bias_hh_l0 torch.bfloat16
model.encoder.latent_projection.weight torch.bfloat16
model.encoder.latent_projection.bias torch.bfloat16
model.decoder.latent_to_states.weight torch.bfloat16
model.decoder.latent_to_states.bias torch.bfloat16
model.decoder.gru1.weight_ih_l0 torch.bfloat16
model.decoder.gru1.weight_hh_l0 torch.bfloat16
model.decoder.gru1.bias_ih_l0 torch.bfloat16
model.decoder.gru1.bias_hh_l0 torch.bfloat16
model.decoder.gru2.weight_ih_l0 torch.bfloat16
model.d

In [3]:
cddd_enc(['CO', 'CN'])

array([[ 0.453125  , -0.9140625 , -0.421875  , ...,  0.78125   ,
         0.875     , -0.8203125 ],
       [ 0.44921875, -0.9375    , -0.3046875 , ...,  0.82421875,
         0.83984375, -0.7734375 ]], dtype=float32)

In [4]:
import pandas as pd

df = pd.read_csv('data/600k_chembl_filtered.smi.zst', header=None, names=['smiles'])
print(df.info())
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636218 entries, 0 to 636217
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   smiles  636218 non-null  object
dtypes: object(1)
memory usage: 4.9+ MB
None


,smiles
0,CC1(C)CCC(C)(C)c2cc(-c3cccc(-c4ccc(C(=O)O)cc4)...
1,Cc1cc2c(cc1C1=NOC(c3ccc(C(=O)O)cc3)C1)C(C)(C)C...
2,Cc1ccc(-c2ccc(C(=O)O)cc2)cc1-c1ccc2c(c1)C(C)(C...
3,Cc1cc2c(cc1-c1ncc(-c3ccc(C(=O)O)cc3)s1)C(C)(C)...
4,Cc1ccc(-c2ccc(C(=O)O)cc2)cc1-c1cc2c(cc1C)C(C)(...
5,Cc1cc2c(cc1-c1cccc(-c3ccc(C(=O)O)cc3)n1)C(C)(C...
6,CC1(C)CCC(C)(C)c2cc(-c3cccc(-c4ccc(C(=O)O)cc4)...
7,CC1(C)CCC(C)(C)c2cc(-c3ccc4cc(C(=O)O)ccc4c3)ccc21
8,CC(=Cc1ccc(C(=O)O)cc1)c1ccc2c(c1)C(C)(C)CCC2(C)C
9,CC(C)=C(c1ccc(C(=O)O)cc1)c1ccc2c(c1)C(C)(C)CCC...


In [4]:
df.sample(1000).to_csv('data/1k_test.smi', index=False, header=False)

In [5]:
cddd_enc(df.smiles)

Encoding SMILES: 100%|██████████| 9941/9941 [00:31<00:00, 316.18it/s]


array([[ 0.09765625,  0.30078125, -0.13183594, ..., -0.84765625,
        -0.84375   ,  0.12988281],
       [ 0.31640625,  0.65234375, -0.27929688, ..., -0.88671875,
        -0.5234375 ,  0.2890625 ],
       [ 0.359375  ,  0.46875   , -0.19042969, ..., -0.77734375,
        -0.8671875 ,  0.08154297],
       ...,
       [ 0.60546875,  0.76171875, -0.16308594, ..., -0.86328125,
         0.26953125,  0.89453125],
       [ 0.890625  ,  0.18457031, -0.5390625 , ..., -0.58203125,
        -0.33203125,  0.8203125 ],
       [ 0.84765625, -0.04101562, -0.7265625 , ..., -0.56640625,
        -0.43164062,  0.75      ]], dtype=float32)